In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from itertools import combinations as comb
import ntpath
import glob
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re

import bert
from sklearn.model_selection import train_test_split # Split Data

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from transformers import TFBertModel, BertTokenizer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score # Performance Metrics  
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import bert
import transformers
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

In [ ]:
df=pd.read_csv("../input/sms-spam-collection-dataset/spam.csv",delimiter=',',encoding='latin-1')
df.head()

In [ ]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
df.rename(columns={'v1':'label','v2':'text'},inplace=True)

In [ ]:
df['label'].value_counts()

In [ ]:
X = df['text']
le=LabelEncoder()

df['label']=le.fit_transform(df['label'])
y = df['label']
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1", trainable=True)

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME,do_lower_case = True)

def encoder(sentences):
  ids = []
  for sentence in sentences:
    encoding = tokenizer.encode_plus(
    sentence,
    max_length=16,
    truncation = True,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=False)
    ids.append(encoding['input_ids'])
  return ids

#Train test split
train_sents,test_sents, train_labels, test_labels  = train_test_split(X,y,test_size=0.15)

train_ids = encoder(train_sents)
test_ids = encoder(test_sents) 

In [ ]:
train_ids = tf.convert_to_tensor(train_ids)
test_ids = tf.convert_to_tensor(test_ids)
test_labels = tf.convert_to_tensor(test_labels)
train_labels = tf.convert_to_tensor(train_labels)

In [ ]:
bert_encoder = TFBertModel.from_pretrained('bert-base-uncased')
input_word_ids = tf.keras.Input(shape=(16,), dtype=tf.int32, name="input_word_ids")  
embedding = bert_encoder([input_word_ids])
dense = tf.keras.layers.Lambda(lambda seq: seq[:, 0, :])(embedding[0])
dense = tf.keras.layers.Dense(128, activation='relu',kernel_regularizer='l2')(dense)
dense = tf.keras.layers.Dropout(0.2)(dense)   
output = tf.keras.layers.Dense(1, activation='sigmoid',kernel_regularizer='l2')(dense)    

model = tf.keras.Model(inputs=[input_word_ids], outputs=output)  

In [ ]:
model.compile(tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x = train_ids, y = train_labels, epochs = 5, verbose = 1, batch_size = 64, validation_data = (test_ids, test_labels))

In [ ]:
test_pred = sBERT.predict(val_input)
print(" accuracy_score: {}".format(accuracy_score(val_labels,test_pred.round())))

In [ ]:
bert_encoder = Model(input_word_ids, model.get_layer('lambda').output)
bert_encoder.summary()


In [ ]:
train_embed = bert_encoder.predict(train_ids)
test_embed = bert_encoder.predict(test_ids)

In [ ]:
def generate_imp_space(X_train, Y_train, X_test, imp_feature_size, foz):
  X_train=pd.DataFrame(X_train)
  X_test=pd.DataFrame(X_test)
  imp_train_data = X_train.values
  imp_test_data = X_test.values
  Y_train=pd.DataFrame(Y_train)
  d = len(X_train.columns)
  num_class=len(np.unique(Y_train))
  # print("____1_____")
  for i in range(0,imp_feature_size*foz):
    Xindis = np.random.permutation(d)
    for j in range(0,d-(foz-1),foz):  #d/foz kadar doner
      sX = np.random.permutation(num_class)
      s1 = sX[0]
      # print("____2_____")
      s1data = X_train[X_train.index.isin(Y_train[Y_train == str(s1)].index)]
      s2data = X_train[~X_train.index.isin(Y_train[Y_train == str(s1)].index)]
      s1data = s1data.iloc[:,Xindis[j:j+(foz)]]
      s2data = s2data.iloc[:,Xindis[j:j+(foz)]] # s1 vs all other classes, #foz feature
      # print("____3_____")
      s1label = np.ones((s1data.values.shape[0],1),dtype=int)
      s2label = -1*np.ones((s2data.values.shape[0],1),dtype=int)
      Wdata = np.concatenate((s1data,s2data))
      # print("____4_____")
      
      Wdata = x2fx(Wdata)
      Wlabel = np.concatenate((s1label,s2label))
      W = np.matmul(np.matmul(np.linalg.pinv(np.matmul(Wdata.T, Wdata)),Wdata.T),Wlabel)
      
      WW = x2fx(X_train.iloc[:,Xindis[j:j+(foz)]].values)
      imp_train_data = np.concatenate((imp_train_data, np.matmul(WW,W)),axis=1)
      
      TT = x2fx(X_test.iloc[:,Xindis[j:j+(foz)]].values)
      imp_test_data = np.concatenate((imp_test_data, np.matmul(TT,W)),axis=1)
    
  return imp_train_data,imp_test_data
  

In [ ]:
def x2fx(x, model='quadratic'):
    linear = np.c_[np.ones(x.shape[0]), x]
    if model == 'linear':
        return linear
    if model == 'purequadratic':
        return np.c_[linear, x**2]
    interaction = np.array([x[:,i]*x[:,j] for i, j in comb(range(x.shape[1]), 2)]).T
    if model == 'interaction':
        return np.c_[linear, interaction]
    if model == 'quadratic':
        return np.c_[linear, interaction, x**2]
def MajorityVoting(votes):
  results = []
  for i in range(0,votes.shape[1]):
    values, counts = np.unique(votes[:,i], return_counts=True)
    results.append(values[np.argmax(counts)])
  return np.array(results)

In [ ]:
foz=4
imp_feature_size=1
acc_imp=[]

acc_norm=[]
acc_incept=[]
imp_tree_predicts=[]
tree_predicts=[]
train_embed=pd.DataFrame(train_embed)
d=len(train_embed.columns)
for i in range(0,5):
    imp_tr, imp_ts = generate_imp_space(train_embed, train_labels, test_embed, imp_feature_size, foz)
    imp_d = imp_tr.shape[1]
    imp_sel_d = 2* round(math.log2(imp_d)) #feature
    sel_d = 2*round(math.log2(d))
      
    imp_tree = XGBClassifier(max_features=imp_sel_d,max_depth=imp_sel_d, n_estimators=50,random_state=42)
   
    imp_tree.fit(imp_tr, train_labels)
    imp_tree_predicts.append(imp_tree.predict(imp_ts))


    tree = XGBClassifier(max_features=sel_d,max_depth=sel_d, n_estimators=50,random_state=42)
    
    tree.fit(train_embed, train_labels)
    tree_predicts.append(tree.predict(test_embed))
result_imp=MajorityVoting(np.array(imp_tree_predicts))
result_norm=MajorityVoting(np.array(tree_predicts))
acc_imp.append(metrics.accuracy_score(test_labels, result_imp))
acc_norm.append(metrics.accuracy_score(test_labels, result_norm))
#result_norm=encoder.fit_transform(result_norm)
#result_imp=encoder.fit_transform(result_imp)

In [ ]:
print("gb")
print(acc_imp)
print(acc_norm)


In [ ]:
print("rf")
print(acc_imp)
print(acc_norm)


In [ ]:
print("xg")
print(acc_imp)
print(acc_norm)

In [ ]:
print("XGBOOST: ", accuracy_score(test_labels, test_pred1))

In [ ]:
import xgboost as xgb
clf = xgb.XGBClassifier(max_features=100, n_estimators=100, subsample=1, learning_rate=0.07, reg_lambda=0.1, reg_alpha=0.1,\
                       gamma=1)
clf.fit(train_embed, train_labels)
test_pred1 = clf.predict(test_embed).round().astype(int)
print("XGBOOST: ", accuracy_score(val_labels, test_pred1))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfclassifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
rfclassifier.fit(train_embed, train_labels)
test_predrf = rfclassifier.predict(test_embed).round().astype(int)
print("RF: ", accuracy_score(val_labels, test_predrf))